In [1]:
import pickle
from dataclasses import asdict, fields
from pathlib import Path

import igl
import numpy as np
import pyvista as pv

from cardiac_electrophysiology.ulac import parameterization, preprocessing, segmentation, solve

In [2]:
patient_id = "01"
mesh_file = f"../data/processed/patient_{patient_id}/mesh_with_fibers_tags.vtk"
segmentation_file = Path("segmentation.pkl")
feature_tags = segmentation.FeatureTags(MV=0, LAA=1, LIPV=2, LSPV=3, RIPV=4, RSPV=5)

alpha_min = 0.0
alpha_max = 1.0
beta_min = 0.0
beta_max = 1.0
relative_center_lipv = (2 / 3, 2 / 3)
relative_center_lspv = (2 / 3, 1 / 3)
relative_center_ripv = (1 / 3, 2 / 3)
relative_center_rspv = (1 / 3, 1 / 3)
relative_center_laa = (5 / 6, 5 / 6)
radius_pv_inner = 0.1
radius_pv_outer = 0.05
radius_laa = 0.1

In [3]:
vtk_mesh = pv.read(mesh_file)
triangular_mesh = preprocessing.convert_unstructured_to_polydata_mesh(vtk_mesh)
vertices = np.array(triangular_mesh.points)
simplices = preprocessing.convert_pv_cells_to_numpy_cells(triangular_mesh.faces)

with segmentation_file.open("rb") as f:
    (
        boundary_paths,
        roof_connection_paths,
        diagonal_mv_connection_paths,
        pv_inner_outer_connection_paths,
        pv_markers,
        laa_markers,
        mv_markers,
    ) = pickle.load(f)

In [4]:
parameterized_pv_paths = parameterization.parameterize_pv_paths_by_arc_length(
    triangular_mesh, boundary_paths, pv_markers
)
parameterized_laa_path = parameterization.parameterize_laa_path_by_arc_length(
    triangular_mesh, boundary_paths, laa_markers
)
parameterized_mv_path = parameterization.parameterize_mv_path_by_arc_length(
    triangular_mesh, boundary_paths, mv_markers
)
parameterized_roof_connections = parameterization.parameterize_roof_connections(
    triangular_mesh, roof_connection_paths
)
parameterized_pv_inner_outer_paths = parameterization.parameterize_pv_inner_outer_paths(
    triangular_mesh, pv_inner_outer_connection_paths
)
parameterized_pv_roof_corners = parameterization.parameterize_pv_roof_corners(
    parameterized_pv_paths
)
parameterized_pv_sections = parameterization.parameterize_pv_sections(parameterized_pv_paths)

In [ ]:
uac_pv_specs = parameterization.get_uac_pv_specs(
    alpha_min=alpha_min,
    alpha_max=alpha_max,
    beta_min=beta_min,
    beta_max=beta_max,
    relative_center_lipv=relative_center_lipv,
    relative_center_lspv=relative_center_lspv,
    relative_center_ripv=relative_center_ripv,
    relative_center_rspv=relative_center_rspv,
    radius_pv_inner=radius_pv_inner,
    radius_pv_outer=radius_pv_outer,
)
uac_laa_specs = parameterization.get_uac_laa_specs(
    alpha_min=alpha_min,
    alpha_max=alpha_max,
    beta_min=beta_min,
    beta_max=beta_max,
    relative_center_laa=relative_center_laa,
    radius_laa=radius_laa,
)
uac_mv_specs = parameterization.get_uac_mv_specs(
    alpha_min=alpha_min,
    alpha_max=alpha_max,
    beta_min=beta_min,
    beta_max=beta_max,
)
uac_roof_specs = parameterization.get_uac_roof_specs(uac_pv_specs)

In [ ]:
uac_pv_boundaries = parameterization.compute_uac_pv_section_boundaries(
    parameterized_pv_sections, uac_pv_specs
)
uac_roof_boundaries = parameterization.compute_uac_roof_boundaries(
    parameterized_roof_connections, parameterized_pv_roof_corners, uac_roof_specs
)
uac_mv_boundary = parameterization.compute_uac_mv_boundary(parameterized_mv_path, uac_mv_specs)

In [ ]:
boundary_indices, boundary_values = solve.extract_uac_boundary_data(
    asdict(uac_roof_boundaries).keys(), uac_roof_boundaries
)
exterior_mesh, roof_mesh = preprocessing.split_from_enclosing_boundary(
    triangular_mesh, boundary_indices
)
roof_vertices = np.array(roof_mesh.points)
roof_simplices = preprocessing.convert_pv_cells_to_numpy_cells(roof_mesh.cells)
local_boundary_inds = preprocessing.get_local_point_inds(roof_mesh, boundary_indices)
harmonic_mapping = igl.harmonic(
    roof_vertices, roof_simplices, local_boundary_inds, boundary_values, 1
)
mapping_z = np.zeros(roof_vertices.shape[0])
mapping_3d = np.hstack([harmonic_mapping, mapping_z[:, None]])
uac_roof_mesh = pv.PolyData.from_regular_faces(mapping_3d, roof_simplices)


In [ ]:
lscm_mapping, _ = igl.lscm(roof_vertices, roof_simplices, local_boundary_inds, boundary_values)

In [ ]:
uac_pv_meshes = {}
for vein in ["LIPV", "LSPV", "RIPV", "RSPV"]:
    boundary_indices, boundary_values = solve.extract_uac_boundary_data(
        [f"{vein}_inner", f"{vein}_outer"], uac_pv_boundaries
    )

    pv_mesh = triangular_mesh.extract_values(getattr(feature_tags, vein), scalars="anatomical_tags")
    pv_vertices = np.array(pv_mesh.points)
    pv_simplices = preprocessing.convert_pv_cells_to_numpy_cells(pv_mesh.cells)
    local_boundary_inds = preprocessing.get_local_point_inds(pv_mesh, boundary_indices)
    harmonic_mapping = igl.harmonic(
        pv_vertices, pv_simplices, local_boundary_inds, boundary_values, 2
    )
    mapping_z = np.zeros(pv_vertices.shape[0])
    mapping_3d = np.hstack([harmonic_mapping, mapping_z[:, None]])
    uac_pv_meshes[vein] = pv.PolyData.from_regular_faces(mapping_3d, pv_simplices)

In [ ]:
plotter = pv.Plotter(window_size=[800, 800])
plotter.add_mesh(
    uac_roof_mesh,
    color="lightgrey",
    style="wireframe",
)
for pv_mesh in uac_pv_meshes.values():
    plotter.add_mesh(
        pv_mesh,
        color="lightgrey",
        style="wireframe",
    )
plotter.view_xy()
plotter.show()